In [1]:
import pandas as pd
from functools import reduce
import numpy as np

In [2]:
titleBasics = pd.read_table('title.basics.tsv') 
titleCrew = pd.read_table('title.crew.tsv')
titleRatings = pd.read_table('title.ratings.tsv')
titlePrincipals = pd.read_table('title.principals.tsv')

/var/folders/4x/2x0b2jfn10194njsl4kvqqc40000gn/T/ipykernel_83713/3939547360.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titleBasics = pd.read_table('title.basics.tsv')


#Converts titlePrincipals df to only contain the first two actors in each film

In [3]:
actors_df = titlePrincipals[titlePrincipals['category'].isin(['actor', 'actress', 'self'])]

# Step 2: Group by tconst and aggregate actor names into a list
grouped_actors = actors_df.groupby('tconst')['nconst'].apply(list).reset_index()

# Step 3: Ensure exactly 2 columns for actors (actor1, actor2)
max_actors = 2  # Adjust this number if you want more actors per row
actor_columns = [f'actor{i+1}' for i in range(max_actors)]

# Step 4: Convert lists to separate columns
actor_df = pd.DataFrame(grouped_actors['nconst'].apply(lambda x: x[:max_actors] + [None] * (max_actors - len(x))).tolist(),
                        columns=actor_columns)

# Step 5: Combine tconst with actor columns
final_actors = pd.concat([grouped_actors[['tconst']], actor_df], axis=1)

In [4]:
titlePrincipals2 = titlePrincipals.drop("job", axis = 1)
df = titlePrincipals2.drop("characters", axis =1)

#dropping unnecessary columns

#Merges the various datasets into one large df, maintaining exactly one row per movie

In [5]:
def merge_dataframes_on_key(dataframes, key_column):
    """
    Merges multiple dataframes on a common key column.

    :param dataframes: List of dataframes to merge
    :param key_column: Name of the key column on which to merge
    :return: A single merged dataframe
    """
    if not dataframes:
        raise ValueError("The list of dataframes is empty.")
    
    # Ensure all dataframes contain the key column
    for df in dataframes:
        if key_column not in df.columns:
            raise ValueError(f"Key column '{key_column}' not found in one of the dataframes.")
    
    # Merge all dataframes on the key column
    merged_df = reduce(lambda left, right: pd.merge(left, right, on=key_column, how='outer'), dataframes)
    
    return merged_df

merged_df = merge_dataframes_on_key([titleBasics, titleCrew, titleRatings, final_actors], 'tconst')

#merge titleBasics, titleCrew, titleRatings based on tconst

In [6]:
merged_df.sort_values(by='numVotes', ascending=False)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes,actor1,actor2
108643,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama,nm0001104,"nm0000175,nm0001104",9.3,2994771.0,nm0000209,nm0000151
449873,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",nm0634240,"nm0634300,nm0634240,nm0275286,nm0004170",9.0,2974365.0,nm0000288,nm0005132
3015913,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",nm0634240,nm0634240,8.8,2641978.0,nm0000138,nm0330687
133554,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,Drama,nm0000399,"nm0657333,nm0880243",8.8,2421498.0,nm0000093,nm0001570
914512,tt0944947,tvSeries,Game of Thrones,Game of Thrones,0,2011,2019,60,"Action,Adventure,Drama","nm0638354,nm0851930,nm0336241,nm0764601,nm0617...","nm1125275,nm0552333,nm1888967,nm4984276,nm2643...",9.2,2391316.0,nm3592338,nm0227759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11415165,tt9916846,tvEpisode,Episode #3.18,Episode #3.18,0,2009,\N,\N,Drama,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284",NaN,NaN,nm5262613,nm8250913
11415166,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,\N,\N,Drama,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284",NaN,NaN,nm5262613,nm8250913
11415167,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,Drama,nm1485677,"nm9187127,nm1485677,nm9826385,nm1628284",NaN,NaN,nm5262613,nm8250913
11415168,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,Drama,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284",NaN,NaN,nm5262613,nm8250913


#Drop all non-movie titles from the df

In [7]:
merged_df = merged_df[~merged_df['titleType'].isin(['tvSeries', 'tvEpisode', "tvMiniSeries", "videoGame", "video", "tvShort"])]
#dropping everything that isnt a movie, tvMovie, short, or tvSpecial (?)

In [8]:
final = merged_df
final = final.drop("originalTitle", axis = 1)
final = final.drop("endYear", axis = 1)
final = final.drop("writers", axis = 1)
final = final.drop("isAdult", axis = 1)
#final = final.drop("primaryTitle", axis = 1)

#dropping unneeded columns

#Switch to numerical title types

In [9]:
title_type_mapping = {'movie': 1, 'tvSpecial': 2, 'tvMovie': 3, 'short': 4}
final['titleType'] = final['titleType'].map(title_type_mapping)
finalSorted = final
#title type mapping

#Gets exactly one director per movie

In [10]:
finalSorted['directors'] = finalSorted['directors'].astype(str).apply(lambda x: x.split(',')[0] if x != 'nan' else '')
#getting exactly one director per movie

In [22]:
finalSorted.head(10)

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,directors,averageRating,numVotes,actor1,actor2,primaryGenre,rating
0,tt0000001,4.0,Carmencita,1894,1,nm0005690,5.7,2118.0,nm1588970,None,Documentary,6.202054
1,tt0000002,4.0,Le clown et ses chiens,1892,5,nm0721526,5.6,285.0,NaN,NaN,Animation,6.370789
2,tt0000003,4.0,Poor Pierrot,1892,5,nm0721526,6.4,2158.0,NaN,NaN,Animation,6.410285
3,tt0000004,4.0,Un bon bock,1892,12,nm0721526,5.3,183.0,NaN,NaN,Animation,6.375366
4,tt0000005,4.0,Blacksmith Scene,1893,1,nm0005690,6.2,2886.0,nm0443482,nm0653042,Short,6.336142
5,tt0000006,4.0,Chinese Opium Den,1894,1,nm0005690,5.0,204.0,NaN,NaN,Short,6.359266
6,tt0000007,4.0,Corbett and Courtney Before the Kinetograph,1894,1,nm0005690,5.3,897.0,nm0179163,nm0183947,Short,6.245162
7,tt0000008,4.0,Edison Kinetoscopic Record of a Sneeze,1894,1,nm0005690,5.4,2268.0,nm0653028,None,Documentary,6.098076
8,tt0000009,1.0,Miss Jerry,1894,45,nm0085156,5.4,218.0,nm0063086,nm0183823,Romance,6.372330
9,tt0000010,4.0,Leaving the Factory,1895,1,nm0525910,6.8,7834.0,NaN,NaN,Documentary,6.649900


#Separate genres into separate columns. Every movie has at least one genre

In [ ]:
raw_genres = [
    'Drama', 'Action', 'Crime', 'Adventure', 'Comedy', 'Biography', 'Mystery', 'Horror',
    'Sci-Fi', 'Animation', 'Fantasy', 'Documentary', 'Thriller', 'Western', 'Romance',
    'Musical', 'Family', 'Music', 'Film-Noir', 'History', 'Short', 'War', 'Adult',
    'Reality-TV', 'News', 'Sport', 'Talk-Show', 'Game-Show',
    np.nan, 'Drama', 'Sci-Fi', 'Crime', 'Adventure', 'Romance', 'War', 'Mystery',
    'Thriller', 'Animation', 'Horror', 'Music', 'Family', 'Fantasy', 'Sport', 'Biography',
    'Western', 'Musical', 'History', 'Film-Noir', 'Documentary', 'News', 'Short',
    'Game-Show', 'Reality-TV', 'Talk-Show', 'Adult'
]

# Extract the first genre from the genre column
finalSorted['primaryGenre'] = finalSorted['genres'].str.split(',').str[0]

# Drop the original genre column (optional, if you don't need it anymore)
finalSorted2 = finalSorted.drop(columns=['genres'])



# 📚 Define comprehensive genre list (from multiple sources)


# 🔄 Create a unique, sorted list of genres (excluding NaN)
#unique_genres = sorted(set(filter(pd.notna, raw_genres)))

# 🧹 Extract the primary genre (first listed genre)
#final_sorted['primary_genre'] = final_sorted['genres'].str.split(',').str[0]

# 🗑️ Drop the original 'genres' column if no longer needed
#final_sorted = final_sorted.drop(columns=['genres'])


KeyError: 'genres'

#Implements a more sophisicated scoring system, the same system that IMDB currently uses. Scores range from 0-10

In [13]:
M = 5000  # Minimum votes required
C = finalSorted['averageRating'].mean()  # Total mean rating
V = finalSorted['numVotes']
R = finalSorted['averageRating']
finalSortedRated = finalSorted
finalSortedRated['rating'] = (V / (V + M)) * R + (M / (V + M)) * C

#implements a scoring system, same system as IMDB

In [14]:
realfinal = finalSortedRated.sort_values(by='rating', ascending=False)
realfinalPruned = realfinal.head(10000)
#selecting only the top 10,000 movies

In [15]:
#realfinalPruned = realfinalPruned.drop('primaryTitle', axis = 1)
realfinalPruned = realfinalPruned.drop('titleType', axis = 1)
realfinalPruned = realfinalPruned.drop('averageRating', axis = 1)
realfinalPruned = realfinalPruned.drop('numVotes', axis = 1)

In [16]:
# Replace all "\\n", "NaN", and "null" entries with empty strings in the entire dataset
realfinalPruned.replace({'\n': ""}, regex=True, inplace=True)
realfinalPruned.replace(r'\\N', '0', regex=True)


# Additionally, replace any NaN (missing values) with empty strings
realfinalPruned.fillna("", inplace=True)


#One hot encoding genres

In [17]:
realfinalPruned.replace(r'\\N', '0', regex=True)

,tconst,primaryTitle,startYear,runtimeMinutes,directors,actor1,actor2,primaryGenre,rating
108643,tt0111161,The Shawshank Redemption,1994,142,nm0001104,nm0000209,nm0000151,Drama,9.295191
67262,tt0068646,The Godfather,1972,175,nm0000338,nm0000008,nm0000199,Crime,9.193354
449873,tt0468569,The Dark Knight,2008,152,nm0634240,nm0000288,nm0005132,Action,8.995661
161575,tt0167260,The Lord of the Rings: The Return of the King,2003,201,nm0001392,nm0000704,nm0001557,Adventure,8.993709
105596,tt0108052,Schindler's List,1993,195,nm0000229,nm0000553,nm0000146,Biography,8.991416
...,...,...,...,...,...,...,...,...,...
271118,tt0283283,Millennium Mambo,2001,119,nm0396284,nm0795517,nm0304709,Drama,6.763065
9089881,tt4768776,"Right Now, Wrong Then",2015,121,nm0393254,nm1045684,nm1195119,Drama,6.763056
6862449,tt28523950,Chip & Wiley Get Stoned,2023,0,nm14658363,nm14658363,nm14665002,Comedy,6.763042
4555011,tt1705120,Love.net,2011,109,nm3078995,nm0794885,nm1707712,Drama,6.762959


In [18]:
realfinalPruned['runtimeMinutes'] = realfinalPruned['runtimeMinutes'].replace(r'\\N', '0', regex=True)  # Replace with 0


In [19]:
realfinalPruned.to_csv('movieDataset.csv', index=False)